Name: Robert (Bobby) Fisher

CWID: 11187333

In [1]:
# import required modules
from pyspark.mllib.stat import Statistics
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.clustering import KMeans, KMeansModel

In [2]:
# load iris data into a RDD
iris_0 = sc.textFile('hdfs://localhost:9000/user/pistolpete/spark_data/iris.data')

In [3]:
#function to parse each line
def parseRow(x):
    p = x.split(',')
    return [float(p[0]), float(p[1]), float(p[2]), float(p[3]), p[4]]

# create RDD with parsed data
iris_1 = iris_0.map(parseRow)

In [4]:
# create RDD with just the numeric columns
iris_2 = iris_1.map(lambda x: x[0:4])

# compute summary statistics
summary_stats = Statistics.colStats(iris_2)

print('Minimum: ', summary_stats.min())
print('Maximum: ', summary_stats.max())
print('Mean: ', summary_stats.mean())
print('Variance: ', summary_stats.variance())
print()

# compute Pearson correlation
corr_mat = Statistics.corr(iris_2)
print('Correlation Matrix')
print(corr_mat)

Minimum:  [ 4.3  2.   1.   0.1]
Maximum:  [ 7.9  4.4  6.9  2.5]
Mean:  [ 5.84333333  3.054       3.75866667  1.19866667]
Variance:  [ 0.68569351  0.18800403  3.11317942  0.58241432]

Correlation Matrix
[[ 1.         -0.10936925  0.87175416  0.81795363]
 [-0.10936925  1.         -0.4205161  -0.35654409]
 [ 0.87175416 -0.4205161   1.          0.9627571 ]
 [ 0.81795363 -0.35654409  0.9627571   1.        ]]


In [5]:
# prepare RDD for use with regression algorithms
# using LabeledPoint: label, features
iris_reg = iris_2.map(lambda x: LabeledPoint(x[0], x[1:]))

# split data into training and testing sets
iris_train, iris_test = iris_reg.randomSplit([0.6, 0.4], seed = 1234)
print('Train: ', iris_train.count())
print('Test : ', iris_test.count())

Train:  94
Test :  56


In [6]:
# train a linear regression model
iris_lr = LinearRegressionWithSGD.train(
	iris_train, 
	step = 0.1, 
	intercept = True)

# train a random forest model	
iris_rf =  RandomForest.trainRegressor(
	iris_train, categoricalFeaturesInfo={},
	numTrees=5, featureSubsetStrategy="auto",
	impurity='variance', maxDepth=4, 
	maxBins=32, seed = 1234)
	
# train a gradient boosted tree model
iris_gbt = GradientBoostedTrees.trainRegressor(
	iris_train, categoricalFeaturesInfo={}, 
	numIterations=10)

/home/pistolpete/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [7]:
# function to compute MSE
def MSE(rdd):
    mse = rdd.map(lambda x: (x[0] - x[1]) ** 2).mean()
    return mse
	
# create RDDs for original labels and features
iris_test_labs = iris_test.map(lambda x: x.label)
iris_test_feat = iris_test.map(lambda x: x.features)

# score test data using models
pred_lr = iris_lr.predict(iris_test_feat)
pred_rf = iris_rf.predict(iris_test_feat)
pred_gbt = iris_gbt.predict(iris_test_feat)

# create RDDs with predicted values and original values
pred_lr_1  = pred_lr.zip(iris_test_labs)
pred_rf_1  = pred_rf.zip(iris_test_labs)
pred_gbt_1  = pred_gbt.zip(iris_test_labs)

# compute and print MSEs
print('MSE LR: ', MSE(pred_lr_1))
print('MSE RF: ', MSE(pred_rf_1))
print('MSE GBT: ', MSE(pred_gbt_1))

MSE LR:  0.173507857234
MSE RF:  0.13288984083013614
MSE GBT:  0.10976242434294911


In [8]:
# function to print cluster tabulation
def printClusterTab(x):
    for z in x:
        print(z[0], ' --> ', z[1])
    
# RDD with original cluster labels
iris_orig_labs = iris_1.map(lambda x: x[4])

# train k-means model with 3 clusters
iris_kmeans = KMeans.train(iris_2, 3, maxIterations=500, initializationMode="random")

# score data to produce new labels
iris_pred_labs = iris_kmeans.predict(iris_2)

# compute cluster tabulation
iris_labs_1 = iris_orig_labs.zip(iris_pred_labs).map(lambda x: [x, 1])
iris_labs_2 = iris_labs_1.groupBy(lambda x: x[0])
iris_labs_3 = iris_labs_2.map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[0])
printClusterTab(iris_labs_3.collect())


('setosa', 0)  -->  50
('versicolor', 1)  -->  47
('versicolor', 2)  -->  3
('virginica', 1)  -->  14
('virginica', 2)  -->  36
